# Problem Set 2: Two-player Iterated Prisoner's Dilemma
Minho Kang (239742)

## Introduction
In this problem set, I will simulate a two-player Iterated Prisoner's Dilemma (IPD) game using several strategies. The objective of this assignment is to explore how different strategies interact in the context of the prisoner's dilemma, focusing on payoffs, cooperation, and defection patterns. We will test the following strategies:

1. Tit-for-Tat
2. Grim Trigger
3. Always Cooperate
4. Always Defect
5. Probabilistic Strategy
6. Intermediate Punishment Strategy

### The Payoff Matrix
The payoff matrix used for the Iterated Prisoner's Dilemma is as follows:

|               | **Cooperate** | **Defect** |
|---------------|---------------|------------|
| **Cooperate** | 6, 6          | 2, 10      |
| **Defect**    | 10, 2         | 4, 4       |

## Objective
The primary objective of this problem set is to implement and simulate a series of strategies in an iterated version of the Prisoner's Dilemma game and analyze their payoffs and behavior when played against each other.

## Methodology
### Strategies Implemented
1. **Tit-for-Tat**: This strategy cooperates on the first move and then mimics the opponent's previous move.
2. **Grim Trigger**: This strategy cooperates until the opponent defects, after which it defects permanently.
3. **Always Cooperate**: This strategy always cooperates regardless of the opponent's moves.
4. **Always Defect**: This strategy always defects regardless of the opponent's moves.
5. **Probabilistic Strategy**: This strategy cooperates with a given probability `p` and defects with the complementary probability.
6. **Intermediate Punishment**: This strategy cooperates until the opponent defects. If the opponent defects, it will not cooperate for the next `k` rounds but will return to cooperation afterward.

### Simulation Setup
The game was simulated over a series of 10 rounds, where each strategy played against every other strategy, except against itself. For each game, the cumulative payoffs were calculated, and the moves of the players were recorded for visualization.

The number of rounds was chosen to be 10 to balance between observing long-term effects and avoiding excessive computation time. Additionally, the `k` parameter in the Intermediate Punishment strategy was set to 3.
And "gamma" (discount factor) was set to 0.9.


In [1]:
import os
from ps2.strategies import *
from ps2.save_plot import save_plot
from ps2.simulate_game import simulate_game

In [2]:
# create payoff_matrix
payoff_matrix = {
    ("C", "C"): (6, 6),
    ("C", "D"): (2, 10),
    ("D", "C"): (10, 2),
    ("D", "D"): (4, 4),
}

In [3]:
# set a directory to save plots
save_dir = "/Users/minhokang/hertie/agt-fall25/ps2/plots"
os.makedirs(save_dir, exist_ok=True)

## Simulate games and save plot of results

In [4]:
# set the seed for reproducibility
np.random.seed(42)

In [5]:
# create a list of tuple for strategy names and their functions
strategies = [
    ("Tit-for-Tat", tit_for_tat),
    ("Grim Trigger", grim_trigger),
    ("Always Cooperate", always_cooperate),
    ("Always Defect", always_defect),
    ("Probabilistic (p=0.5)", lambda h: probabilistic_strategy(h, p=0.5)), # set the seed for reproducibility
    ("Intermediate Punishment", intermediate_punishment),
]

# simulate and save results for all strategy combinations
payoffs = []  # to store payoff results
cooperation_rates = {} # to store cooperation rates for each strategy pair

for i in range(len(strategies)):
    for j in range(i, len(strategies)):
        strategy1_name, strategy1 = strategies[i]
        strategy2_name, strategy2 = strategies[j]

        # simulate the game
        player1_moves, player2_moves, player1_payoff, player2_payoff = simulate_game(
            player1_strategy=strategy1,
            player2_strategy=strategy2,
            rounds=10,
            payoff_matrix=payoff_matrix,
            gamma=0.9, # discount factor
        )

        # save the plot and calculate payoffs
        save_plot(
            player1_strategy_name=strategy1_name,
            player2_strategy_name=strategy2_name,
            player1_moves=player1_moves,
            player2_moves=player2_moves,
            player1_payoff=player1_payoff,
            player2_payoff=player2_payoff,
            save_dir=save_dir,
        )

        # store the payoffs for later analysis
        payoffs.append((strategy1_name, strategy2_name, player1_payoff, player2_payoff))

        # calculate cooperation rates
        cooperation_rate1 = sum(player1_moves) / len(player1_moves)
        cooperation_rate2 = sum(player2_moves) / len(player2_moves)
        cooperation_rates[(strategy1_name, strategy2_name)] = (cooperation_rate1, cooperation_rate2)

# find plots of each strategy results from the below link
# https://github.com/minhokg/agt-fall25/tree/main/ps2/plots

## Summary of games (total payoff and cooperation rates for each game)

In [6]:
## 1. payoff
print("Payoffs Summary:")
for payoff in payoffs:
    print(f"{payoff[0]} vs {payoff[1]}: {payoff[2]} - {payoff[3]}")


Payoffs Summary:
Tit-for-Tat vs Tit-for-Tat: 39.079293594 - 39.079293594
Tit-for-Tat vs Grim Trigger: 39.079293594 - 39.079293594
Tit-for-Tat vs Always Cooperate: 39.079293594 - 39.079293594
Tit-for-Tat vs Always Defect: 24.052862396000002 - 32.052862396
Tit-for-Tat vs Probabilistic (p=0.5): 31.476730596000003 - 37.254305796000004
Tit-for-Tat vs Intermediate Punishment: 39.079293594 - 39.079293594
Grim Trigger vs Grim Trigger: 39.079293594 - 39.079293594
Grim Trigger vs Always Cooperate: 39.079293594 - 39.079293594
Grim Trigger vs Always Defect: 24.052862396000002 - 32.052862396
Grim Trigger vs Probabilistic (p=0.5): 46.202374590000005 - 26.803024998000005
Grim Trigger vs Intermediate Punishment: 39.079293594 - 39.079293594
Always Cooperate vs Always Cooperate: 39.079293594 - 39.079293594
Always Cooperate vs Always Defect: 13.026431198000001 - 65.13215599
Always Cooperate vs Probabilistic (p=0.5): 29.082277154000003 - 49.076310034
Always Cooperate vs Intermediate Punishment: 39.0792935

In [10]:
# Initialize variables to track the highest payoffs
highest_payoff_player1 = max(payoffs, key=lambda x: x[2])  # Highest payoff for Player 1
highest_payoff_player2 = max(payoffs, key=lambda x: x[3])  # Highest payoff for Player 2

# Combine both highest payoffs to find the highest overall payoff
if highest_payoff_player1[2] > highest_payoff_player2[3]:
    highest_combined_payoff = highest_payoff_player1
    highest_combined_value = highest_payoff_player1[2]  # Player 1's payoff
else:
    highest_combined_payoff = highest_payoff_player2
    highest_combined_value = highest_payoff_player2[3]  # Player 2's payoff

# Print the highest combined payoff
print(f"Highest Combined Payoff: {highest_combined_payoff[0]} vs {highest_combined_payoff[1]} with Total Payoff {highest_combined_value} for {highest_combined_payoff[1]}")

Highest Combined Payoff: Always Cooperate vs Always Defect with Total Payoff 65.13215599 for Always Defect


In [7]:
## 2. cooperation rates
print("\nCooperation Rates:")
for (strategy1_name, strategy2_name), (cooperation_rate1, cooperation_rate2) in cooperation_rates.items():
    print(f"{strategy1_name} vs {strategy2_name}: {cooperation_rate1:.2f} - {cooperation_rate2:.2f}")


Cooperation Rates:
Tit-for-Tat vs Tit-for-Tat: 1.00 - 1.00
Tit-for-Tat vs Grim Trigger: 1.00 - 1.00
Tit-for-Tat vs Always Cooperate: 1.00 - 1.00
Tit-for-Tat vs Always Defect: 0.10 - 0.00
Tit-for-Tat vs Probabilistic (p=0.5): 0.50 - 0.40
Tit-for-Tat vs Intermediate Punishment: 1.00 - 1.00
Grim Trigger vs Grim Trigger: 1.00 - 1.00
Grim Trigger vs Always Cooperate: 1.00 - 1.00
Grim Trigger vs Always Defect: 0.10 - 0.00
Grim Trigger vs Probabilistic (p=0.5): 0.20 - 0.70
Grim Trigger vs Intermediate Punishment: 1.00 - 1.00
Always Cooperate vs Always Cooperate: 1.00 - 1.00
Always Cooperate vs Always Defect: 1.00 - 0.00
Always Cooperate vs Probabilistic (p=0.5): 1.00 - 0.60
Always Cooperate vs Intermediate Punishment: 1.00 - 1.00
Always Defect vs Always Defect: 0.00 - 0.00
Always Defect vs Probabilistic (p=0.5): 0.00 - 0.50
Always Defect vs Intermediate Punishment: 0.00 - 0.10
Probabilistic (p=0.5) vs Probabilistic (p=0.5): 0.60 - 0.40
Probabilistic (p=0.5) vs Intermediate Punishment: 0.60 -

In [8]:
# Here is the high cooperation pairs (cooperation 100%)
high_cooperation_pairs = [
    (strategy1_name, strategy2_name, cooperation_rate1, cooperation_rate2)
    for (strategy1_name, strategy2_name), (cooperation_rate1, cooperation_rate2) in cooperation_rates.items()
    if cooperation_rate1 == 1.0 and cooperation_rate2 == 1.0
]

print("High Cooperation Strategy Pairs:")
for pair in high_cooperation_pairs:
    print(f"{pair[0]} vs {pair[1]}: Cooperation Rates: {pair[2]:.2f} - {pair[3]:.2f}")


High Cooperation Strategy Pairs:
Tit-for-Tat vs Tit-for-Tat: Cooperation Rates: 1.00 - 1.00
Tit-for-Tat vs Grim Trigger: Cooperation Rates: 1.00 - 1.00
Tit-for-Tat vs Always Cooperate: Cooperation Rates: 1.00 - 1.00
Tit-for-Tat vs Intermediate Punishment: Cooperation Rates: 1.00 - 1.00
Grim Trigger vs Grim Trigger: Cooperation Rates: 1.00 - 1.00
Grim Trigger vs Always Cooperate: Cooperation Rates: 1.00 - 1.00
Grim Trigger vs Intermediate Punishment: Cooperation Rates: 1.00 - 1.00
Always Cooperate vs Always Cooperate: Cooperation Rates: 1.00 - 1.00
Always Cooperate vs Intermediate Punishment: Cooperation Rates: 1.00 - 1.00
Intermediate Punishment vs Intermediate Punishment: Cooperation Rates: 1.00 - 1.00


## Conclusion

Here are the keynotes for the analysis:

1. For **Grim Trigger**, **Tit-for-Tat**, and **Intermediate Punishment**, when these strategies play against each other, the cooperation rates are always 100%. This indicates that these strategies are well-suited for cooperation when paired with each other.
2. **"Always Defect" against "Always Cooperate"** results in the highest total payoff for Player 2. This is because **"Always Defect"** exploits the cooperative nature of **"Always Cooperate"** and consistently maximizes Player 2's payoff while minimizing Player 1's payoff.
3. **"Always Cooperate"** maintains a high cooperation rate (1.00 for both players) and results in the best cooperation behavior when playing against cooperative strategies like **Tit-for-Tat**, **Grim Trigger**, and **Intermediate Punishment**.
4. **"Tit-for-Tat"** and **"Grim Trigger"** also achieve high cooperation rates against **"Always Cooperate"** and other cooperative strategies. However, **Tit-for-Tat** is more forgiving than **Grim Trigger**, as **Grim Trigger** will defect permanently after a single defection, while **Tit-for-Tat** will only defect once the opponent defects and then revert back to cooperation.
5. **"Probabilistic (p=0.5)"** shows mixed cooperation rates. It cooperates around 50% of the time, but depending on its opponent, the cooperation rate can vary (e.g., it cooperates less with **"Always Defect"** and more with cooperative strategies).
6. The **Intermediate Punishment Strategy** is an interesting case, as it balances cooperation and punishment. It returns to cooperation after **k** rounds of defection when the opponent defects. It works well in maintaining cooperation with cooperative strategies and has a high cooperation rate when playing against strategies like **Tit-for-Tat** and **Always Cooperate**.

### Final Insights:
- **Cooperation is sustainable** when both players are willing to reciprocate cooperation (like in **Tit-for-Tat**, **Grim Trigger**, and **Intermediate Punishment**).
- **Defection** is a dominant strategy for **Always Defect**, but it often results in a lower overall payoff, especially when paired against cooperative strategies.
- **Tit-for-Tat** is one of the most balanced strategies, performing well against both cooperative and defecting opponents.
- **Intermediate Punishment** can serve as a middle ground, punishing defections temporarily and returning to cooperation, ensuring a good balance between cooperation and defense against exploitation.

This analysis reveals that cooperative strategies like **Tit-for-Tat** and **Grim Trigger** are highly effective in sustaining cooperation, while **Always Defect** is successful in exploiting **Always Cooperate**. The **Intermediate Punishment Strategy** offers a way to sustain cooperation while adding a corrective measure for defection.

---
